# Implement the Architecture of AlenNet network

In [1]:
from tensorflow import keras

In [3]:
from keras.layers import Conv2D, MaxPooling2D, Input, ReLU, BatchNormalization, Dense, Flatten, Concatenate, Dropout

In [5]:
input = Input(shape=(227, 227, 3))

# first set of convolutions
conv_a_down = Conv2D(filters=48, strides=4, kernel_size=11, padding='valid')(input)
batchnorm_a_down = BatchNormalization(axis=-1)(conv_a_down)
out_a_down = ReLU()(batchnorm_a_down)
maxpool_a_down = MaxPooling2D(pool_size=3, strides=2)(out_a_down)

conv_a_up = Conv2D(filters=48, strides=4, kernel_size=11, padding='valid')(input)
batchnorm_a_up = BatchNormalization(axis=-1)(conv_a_up)
out_a_up = ReLU()(batchnorm_a_up)
maxpool_a_up = MaxPooling2D(pool_size=3, strides=2)(out_a_up)

# second set of convolutions
conv_b_down = Conv2D(filters=128, strides=1, kernel_size=5, padding='same')(maxpool_a_down)
batchnorm_b_down = BatchNormalization(axis=-1)(conv_b_down)
out_b_down = ReLU()(batchnorm_b_down)
maxpool_b_down = MaxPooling2D(pool_size=3, strides=2)(out_b_down)

conv_b_up = Conv2D(filters=128, strides=1, kernel_size=5, padding='same')(maxpool_a_up)
batchnorm_b_up = BatchNormalization(axis=-1)(conv_b_up)
out_b_up = ReLU()(batchnorm_b_up)
maxpool_b_up = MaxPooling2D(pool_size=3, strides=2)(out_b_up)

# third set of convolutions; uses both up and down streams 
input_c = Concatenate()([maxpool_b_down, maxpool_b_up])
conv_c_down = Conv2D(filters=192, strides=1, kernel_size=3, padding='same', activation='relu')(input_c)
conv_c_up = Conv2D(filters=192, strides=1, kernel_size=3, padding='same', activation='relu')(input_c)

# fourth layer of convolutions
conv_d_down = Conv2D(filters=192, strides=1, kernel_size=3, padding='same', activation='relu')(conv_c_down)
conv_d_up = Conv2D(filters=192, strides=1, kernel_size=3, padding='same', activation='relu')(conv_c_up)

# fifth layer of convolutions
conv_e_down = Conv2D(filters=128, strides=1, kernel_size=3, padding='same', activation='relu')(conv_d_down)
conv_e_up = Conv2D(filters=128, strides=1, kernel_size=3, padding='same', activation='relu')(conv_d_up)
maxpool_e_down = MaxPooling2D(pool_size=3, strides=2)(conv_e_down)
maxpool_e_up = MaxPooling2D(pool_size=3, strides=2)(conv_e_up)

# sixth layer: concatenate up-down streams, flatten and use dense layer
input_f = Concatenate()([maxpool_e_down, maxpool_e_up])
input_f_flat = Flatten()(input_f)
dense_f_up = Dense(2048, activation='relu')(input_f_flat)
dropout_f_up = Dropout(rate=0.5)(dense_f_up)
dense_f_down = Dense(2048, activation='relu')(input_f_flat)
dropout_f_down = Dropout(rate=0.5)(dense_f_down)

# seven layer: concatenate up-down streams and use dense layer
input_g = Concatenate()([dropout_f_down, dropout_f_up])
dense_g_up = Dense(2048, activation='relu')(input_g)
dropout_g_up = Dropout(rate=0.5)(dense_g_up)
dense_g_down = Dense(2048, activation='relu')(input_g)
dropout_g_down = Dropout(rate=0.5)(dense_g_down)

# eighth layer: concatente and use dense
input_h = Concatenate()([dropout_g_down, dropout_g_up])
dense_h = Dense(1000, activation='softmax')(input_h)

AlexNet = keras.Model(inputs = input, outputs = dense_h)
AlexNet.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1       │ (None, 227, 227,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_4 (Conv2D)   │ (None, 55, 55,    │     17,472 │ input_layer_1[0]… │
│                     │ 48)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_5 (Conv2D)   │ (None, 55, 55,    │     17,472 │ input_layer_1[0]… │
│                     │ 48)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 55, 55,    │        192 │ conv2d_4[0][0]    │
│ (BatchNormalizatio… │ 48)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 55, 55,    │        192 │ conv2d_5[0][0]    │
│ (BatchNormalizatio… │ 48)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_4 (ReLU)      │ (None, 55, 55,    │          0 │ batch_normalizat… │
│                     │ 48)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_5 (ReLU)      │ (None, 55, 55,    │          0 │ batch_normalizat… │
│                     │ 48)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_4     │ (None, 27, 27,    │          0 │ re_lu_4[0][0]     │
│ (MaxPooling2D)      │ 48)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_5     │ (None, 27, 27,    │          0 │ re_lu_5[0][0]     │
│ (MaxPooling2D)      │ 48)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_6 (Conv2D)   │ (None, 27, 27,    │    153,728 │ max_pooling2d_4[… │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_7 (Conv2D)   │ (None, 27, 27,    │    153,728 │ max_pooling2d_5[… │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 27, 27,    │        512 │ conv2d_6[0][0]    │
│ (BatchNormalizatio… │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 27, 27,    │        512 │ conv2d_7[0][0]    │
│ (BatchNormalizatio… │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_6 (ReLU)      │ (None, 27, 27,    │          0 │ batch_normalizat… │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_7 (ReLU)      │ (None, 27, 27,    │          0 │ batch_normalizat… │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_6     │ (None, 13, 13,    │          0 │ re_lu_6[0][0]     │
│ (MaxPooling2D)      │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_7     │ (None, 13, 13,    │          0 │ re_lu_7[0][0]   

 Total params: 60,966,632 (232.57 MB)

 Trainable params: 60,965,928 (232.57 MB)

 Non-trainable params: 704 (2.75 KB)